# Evolving Game Strategies - Preliminary Report
Authors: Subhash Gubba and Kai Levy
## Abstract
We will investigate evolutionary game theory strategies and their implementations to see which ones emerge dominant. We intend to use agent-based modeling (with the help of the Mesa python ABM library) to investigate it. Our first approach will be to mimic the “Evolution of Ethnocentrism” experiment and try various ways to extend it in small ways. A possible reach extension of this project could include implementing a genetic algorithm to see which aspects of strategies might appear in successful ones.

## Experiment 1 - General Implementation
Here we attempt to replicate the agent-based ethnicity model proposed by Hartshorn, Kaznatcheev, and Thomas Shultz.

**Q** - Does our implementation of the model perform similarly to the original model? Does the number of agents for each behavior over time resemble that of the original model?

**M** - We utilize the Mesa ABM library to create our framework for the agent-based model. We implement the steps of the model including reproduction and death of agents. We then run the model and collect the data for analysis.

**R** - see the results below:

In [ ]:
import pandas as pd
from itertools import chain, combinations
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
sys.path.append(os.path.abspath('../code'))
from ethno import EthnoModel, EthnoAgent, BEHAVIOR_KEY

We begin by initializing the model with all behaviors ("Ethnocentric","Humanitarian", "Selfish", "Traitor") and run for around 500 steps.

In [ ]:
m = EthnoModel(5,50,50,1,0.005, [0b00, 0b01, 0b10, 0b11], max_iters=1000)

In [ ]:
m.run_model()

In [ ]:
data = m.datacollector.get_model_vars_dataframe()

In [ ]:
ax = data[["Humanitarian","Traitor", "Ethnocentric", "Selfish"]].plot()
ax.set_title("Population and Behavior Over Time")
ax.set_xlabel("Step")
ax.set_ylabel("Number of Agents")
_ = ax.legend(bbox_to_anchor=(1.35, 1.025))

The original experiment that we recreated:
![Original Experiment Behavior Timeseries](./images/jasss_report_timescale.png)

**I** - The results very closely resemble that of the original model's findings: Ethnocentrism and Humanitarianism are the leaders in the early stages, with ethnocentrism dominating by the 500th (or so) step. Traitorish is the worst performing trait, then Selfish, but neither of them die out completely.

## Experiment 2 - Varying Behaviors Validation
The original paper explores how different combinations of behaviors in the model effect the numbers.

**Q** - Does our model perform the same way the original model does when behaviors are variably included?

**M** - We use the same model implemented, and run it for 2000 steps for each combination of variables, and then observe the average behavior counts for the last 100 steps of the simulation.

**R** - see the results below. This first table is from the original paper. The second table is the DataFrame that depicts our results.

#### Table from Paper - Varied Behavior Inclusion
![Allowed Behavior Mean Agent Counts](./images/meanagentstable.PNG)

A powerset defines all the combinations of behaviors we want to run.

In [ ]:
def powerset(orig_list):
    new_list = list([y for y in orig_list])
    return [list(x) for x in list(chain.from_iterable(combinations(new_list, item) for item in range(1,len(new_list)+1)))]

In [ ]:
behavior_combs = powerset([0b00,0b01,0b10,0b11])
print(behavior_combs)

In [ ]:
behavior_labels = [[BEHAVIOR_KEY[y] for y in x] for x in behavior_combs]

In [ ]:
behavior_labels

In [ ]:
behavior_counts_dict = {}
counter = 0
S_sim_count = 0
T_sim_count = 0
E_sim_count = 0
H_sim_count = 0
for b_list in behavior_combs:
    for _ in range(10):
        m = EthnoModel(5,50,50,1,0.005, b_list, max_iters=2000)
        m.run_model()
        data = m.datacollector.get_model_vars_dataframe()
        S_sim_count += data.Selfish[1900:].get_values().mean()
        T_sim_count += data.Traitor[1900:].get_values().mean()
        E_sim_count += data.Ethnocentric[1900:].get_values().mean()
        H_sim_count += data.Humanitarian[1900:].get_values().mean()
    behavior_counts_dict[''.join(behavior_labels[counter])] = [S_sim_count/10.0,T_sim_count/10.0,E_sim_count/10.0,H_sim_count/10.0]
    counter+=1
    
    

In [ ]:
behavior_counts_dict

We now have a set of results for the mean counts of each behavior. We can also compare this with the original table.

In [ ]:
behavior_counts_table = pd.DataFrame(behavior_counts_dict).transpose()
behavior_counts_table.columns = ['S','T','E','H']

In [ ]:
newtemp = behavior_counts_table[['E','H','S','T']]
newtemp.reindex(['STEH','STH','TEH','SEH','STE','ST','TH','SH','TE','SE','EH','E','H','S','T'])

In [ ]:
newtemp['E %'] = (newtemp['E']/newtemp.sum(1,0))*100
newtemp['H %'] = (newtemp['H']/newtemp.sum(1,0))*100
newtemp['S %'] = (newtemp['S']/newtemp.sum(1,0))*100
newtemp['T %'] = (newtemp['T']/newtemp.sum(1,0))*100
newtemp[['E %','H %','S %','T %']].reindex(['STEH','STH','TEH','SEH','STE','ST','TH','SH','TE','SE','EH','E','H','S','T'])

We can look at the the two tables to see what percentages of the total population they were, and then observe the differences in those percentages.

In [ ]:
orig_model_results = {'E': [100.0, 0.0, 0.0, 0.0],
 'EH': [84.19, 15.81, 0.0, 0.0],
 'H': [0.0, 100.0, 0.0, 0],
 'S': [0.0, 0.0, 100.0, 0.0],
 'SE': [90.1, 0.0, 9.99, 0.0],
 'SEH': [74.97, 17.17, 7.86, 0.0],
 'SH': [0.0, 91.39, 8.61, 0.0],
 'ST': [0.0, 0.0, 66.12, 33.88],
 'STE': [89.08, 0.0, 8.88, 2.04],
 'STEH': [74.78,14.48,7.77,2.97],
 'STH': [0.0, 83.77, 7.04, 9.19],
 'T': [0.0, 0.0, 0.0, 100.0],
 'TE': [97.91, 0.0, 0.0, 2.01],
 'TEH': [82.35, 15.25, 0.0, 2.41],
 'TH': [0.0, 88.72, 0.0, 11.28]}

orig_model_table = pd.DataFrame(orig_model_results).transpose()
orig_model_table.columns = ['E %', 'H %', 'S %', 'T %']
orig_model_table.reindex(['STEH','STH','TEH','SEH','STE','ST','TH','SH','TE','SE','EH','E','H','S','T'])
orig_model_table - newtemp[['E %','H %','S %','T %']]

**I** - Our results above seem to match relatively closely with the table from the paper. The differences in percentages depicted above are not very large with the exception of the simulation where only selfish and traitorous behvaiors were included. Judging based on this fact, it should be safe to say that our implementation of the model is accurate to that of the original one.

![Allowed Behavior Mean Agent Counts](./images/meanagentstable.PNG)

### Additional extensions
We have not yet designed further experiments, because we would like to extend the model in ways that the papers we referenced have not tried yet. Below are a few ideas that we have brainstormed about potential model extensions:
- Allowing for "alliances", agents which will cooperate with some agents of other tags.
- Making tags more ambiguous, agents cannot tell for certain what tag another agent exhibits
- Iterative game playing, with more complex strategies
- Aging and lifespan (not a complete extension, but may be useful)

### Learning Goals

#### Subhash:
I would like to gain a better understand of how agent based models are implemented and evolved. I want to learn what types of evaluation metrics matter for our experiments and explore how this project could be applied to other fields as well.

Working through this implementation and performing the validation experiments has helped me achieve this goal. I have yet to see how we might apply this to other fields.

#### Kai:
I have made progress towards my learning goal in the process of implementing the ethnocentrism model. The experience of reading through a paper in order to recreate it's results was very valuable for me.

For the rest of the project, I would like to do more exploration with extensions (as opposed copying existing experiments). In particular, I want to come up interesting extensions that are not so complex that they muddle insights. I want to ask questions that can be compared to human behavior (like original question of ethnocentrism), and hopefully answer them with experiments in our model.

## Bibliography
- Ross Hammond, Robert Axelrod. [“The Evolution of Ethnocentrism” (2006).](
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.576.4696&rep=rep1&type=pdf.)
  *Investigates prisoner’s dilemma on a grid, with four basic tags and linked behaviors. Simulates the agents by playing the game one-off, with results having implications on individual reproductivity. Demonstrates that in-group favoritism can emerge as beneficial for groups, even when individual cooperation is costly.*


- Max Hartshorna, Artem Kaznatcheeva, Thomas Shultz. [“The Evolutionary Dominance of Ethnocentric Cooperation” (2013).](http://jasss.soc.surrey.ac.uk/16/3/7.html)
*Replicates the experiment from Hammond, Axelrod (2006). Investigates different “worlds” where certain behaviors may or may not be present, and demonstrates that humanitarianism becomes dominant in the absence of ethnocentrism, but ethnocentrism dominates otherwise.*


- Jennifer Golbeck. ["Evolving Strategies for the Prisoner’s Dilemma" (2002).](https://www.cs.umd.edu/~golbeck/downloads/JGolbeck_prison.pdf)
*This paper applies a genetic algorithm to the Prisoner’s Dilemma and evaluates the resulting winning strategies against two well-known effective strategies: Pavlov and Tit-for-tat. Each round of the algorithm is a series of multiple games with randomly strategized players, and the emerging strategies show aspects of the two control strategies.*


- Wikipedia. ["List of games in game theory".](https://en.wikipedia.org/wiki/List_of_games_in_game_theory)
*Contains a list of games for which we may apply agent-based models and evolutionary strategies in game theory.*